# Реализация ASPP сети
В этом уроке мы создадим еще еще одну сегментационную FCN модель на основе блока ASPP (Atrous Spatial Pyramid Pooling), который основан на дилатационных свёртках (Atrous Conv, Dilated Conv)

### Загрузка необходимых библиотек
Здесь мы загружаем различне библиотеки, включая TensoFlow.


In [1]:
import numpy as np

import tensorflow as tf
print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.15.0


### Создание ASPP модели
В этом примере мы будем создавать модель `ASPPNet` через собственный класс, наследованный от `tf.keras.Model`. Кроме того, будем использовать возможность модульного создания нейросетей. Сначала создадим отдельно модель для ASPP блока (блок, который содержит только дилатационные свёртки и применяется между энкодером и декодером), которая по сути будет являться самостоятельным слоем. А затем создадим финальную модель `ASPPNet`, в который `ASPPBlock` будет использоваться как слой между Энкодером и Декодером.

В ASPP блоке (`ASPPBlock`) ко входному тензору параллельно применяются обычная свёртка 1x1 и несколько дилатационных свёрток. Затем все эти резульаты конкатенируются. Далее "перемешиваем" каналы полученного тензора с помощью еще одной свёртки 1x1.

В финальной ASPP модели (`ASPPNet`) в качестве Энкодера будем использовать стандартные свёртки и пулинги. Далее в боттлнеке (в середине сети) применим слой ASPPBlock. А в Декодере будем использовать более простую архитектуру: меньше свёрток, повышение размерности с помощью двух билинейных интерполяций (`tf.image.resize`). Кроме того, в сети присутствуют две проброшенные связи.

In [2]:
class ASPPBlock(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=6, padding='same', activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=12, padding='same', activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=18, padding='same', activation='relu')
        self.conv5 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu')

    def call(self, inp, is_training=False):
        out1 = self.conv1(inp)
        out2 = self.conv2(inp)
        out3 = self.conv3(inp)
        out4 = self.conv4(inp)
        out = tf.concat([out1, out2, out3, out4], axis=3)
        out = self.conv5(out)
        return out

class ASPPNet(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')
        self.conv5 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv6 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv7 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv8 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv9 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv10 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')

        self.conv11 = tf.keras.layers.Conv2D(48, (1, 1), padding='same', activation='relu')
        self.conv12 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv13 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv14 = tf.keras.layers.Conv2D(1, (1, 1), padding='same', activation=None)

        self.maxpool = tf.keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same')

        self.aspp = ASPPBlock()

    def call(self, x):

        out = self.conv1(x)
        out = self.conv2(out)
        out = self.maxpool(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.maxpool(out)
        out = self.conv5(out)
        out = self.conv6(out)
        out_enc_mid = out
        out = self.maxpool(out)
        out = self.conv7(out)
        out = self.conv8(out)
        out = self.maxpool(out)
        out = self.conv9(out)
        out = self.conv10(out)

        out = self.aspp(out)

        out = tf.image.resize(out, tf.shape(out_enc_mid)[1:3], tf.image.ResizeMethod.BILINEAR)

        out_enc_mid = self.conv11(out_enc_mid)

        out = tf.concat([out, out_enc_mid], axis=3)

        out = self.conv12(out)
        out = self.conv13(out)
        out = self.conv14(out)

        out = tf.image.resize(out, tf.shape(x)[1:3], tf.image.ResizeMethod.BILINEAR)
        out = tf.nn.sigmoid(out)
        return out

model = ASPPNet()

### Задания


**[ЗАДАНИЕ 1]** Вопрос: почему в ASPPNet в первой билинейной интерполяции (`tf.image.resize`) в качестве желаемого выходного размера тензора мы используем размер тензора `out_enc_mid`?

В модели ASPPNet используется первая билинейная интерполяция с размером тензора out_enc_mid по следующим причинам:

Восстановление пространственного разрешения:
В первой билинейной интерполяции происходит увеличение разрешения выходного тензора после блока ASPP до размера, совпадающего с разрешением промежуточного выхода out_enc_mid. Это необходимо для корректного объединения информации из разных уровней (skip connections).

Skip connections:
Архитектура модели предполагает использование skip connections для соединения информации из предыдущих уровней свёрточных слоёв (encoder) с последующими уровнями (decoder). Такие соединения позволяют объединить контекстную информацию из более глубоких слоёв с более детальной информацией из поверхностных слоёв, что улучшает сегментационные возможности модели.

Слияние признаков:
Для эффективного объединения информации из разных уровней модели важно, чтобы тензоры, которые объединяются через слой Concatenate, имели одинаковое разрешение. Таким образом, выходной тензор после блока ASPP интерполируется до разрешения out_enc_mid, что позволяет корректно объединить его с out_enc_mid в следующем шаге.

Обоснование
ASPP блок:
Обрабатывает тензор для извлечения контекстной информации с использованием различных уровней дилатации (dilation rates). После этого необходимо вернуть пространственное разрешение тензора обратно для дальнейшего объединения с признаками из out_enc_mid.

Объединение признаков:
После билинейной интерполяции до разрешения out_enc_mid, выход блока ASPP объединяется с этим промежуточным выходом. Это помогает модели лучше восстанавливать детали в процессе декодирования.


**[ЗАДАНИЕ 2]** Реализуйте пайплайн обучения для модели ASPPNet: подготовка данных, лосс, обучение, тестирование. Используйте материалы из предыдущего практического урока. Обучите модель и сравните время обучения с временем обучения более простых версий FCN из предыдущих уроков.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time

# Функция для загрузки и предобработки данных
def load_data(image_paths, mask_paths, img_size):
    images = [tf.image.resize(tf.image.decode_png(tf.io.read_file(img_path), channels=3), img_size) / 255.0 for img_path in image_paths]
    masks = [tf.image.resize(tf.image.decode_png(tf.io.read_file(mask_path), channels=1), img_size) / 255.0 for mask_path in mask_paths]
    return np.array(images), np.array(masks)

# Загрузка данных (пример с путями к данным)
image_paths = ['path/to/images/1.png', 'path/to/images/2.png', ...]
mask_paths = ['path/to/masks/1.png', 'path/to/masks/2.png', ...]
img_size = (256, 256)

images, masks = load_data(image_paths, mask_paths, img_size)
x_train, x_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)


def dice_coefficient(y_true, y_pred):
    smooth = 1e-7
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)


class ASPPBlock(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=6, padding='same', activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=12, padding='same', activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=18, padding='same', activation='relu')
        self.conv5 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu')

    def call(self, inp, is_training=False):
        out1 = self.conv1(inp)
        out2 = self.conv2(inp)
        out3 = self.conv3(inp)
        out4 = self.conv4(inp)
        out = tf.concat([out1, out2, out3, out4], axis=3)
        out = self.conv5(out)
        return out

class ASPPNet(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')
        self.conv5 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv6 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv7 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv8 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv9 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv10 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')

        self.conv11 = tf.keras.layers.Conv2D(48, (1, 1), padding='same', activation='relu')
        self.conv12 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv13 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv14 = tf.keras.layers.Conv2D(1, (1, 1), padding='same', activation=None)

        self.maxpool = tf.keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same')

        self.aspp = ASPPBlock()

    def call(self, x):

        out = self.conv1(x)
        out = self.conv2(out)
        out = self.maxpool(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.maxpool(out)
        out = self.conv5(out)
        out = self.conv6(out)
        out_enc_mid = out
        out = self.maxpool(out)
        out = self.conv7(out)
        out = self.conv8(out)
        out = self.maxpool(out)
        out = self.conv9(out)
        out = self.conv10(out)

        out = self.aspp(out)

        out = tf.image.resize(out, tf.shape(out_enc_mid)[1:3], tf.image.ResizeMethod.BILINEAR)

        out_enc_mid = self.conv11(out_enc_mid)

        out = tf.concat([out, out_enc_mid], axis=3)

        out = self.conv12(out)
        out = self.conv13(out)
        out = self.conv14(out)

        out = tf.image.resize(out, tf.shape(x)[1:3], tf.image.ResizeMethod.BILINEAR)
        out = tf.nn.sigmoid(out)
        return out

model = ASPPNet()


model.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coefficient])

# Настройка для замера времени обучения
start_time = time.time()

history = model.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    epochs=50,
                    batch_size=16)

training_time = time.time() - start_time
print(f'Training time: {training_time} seconds')


loss, dice = model.evaluate(x_val, y_val)
print(f'Validation Loss: {loss}')
print(f'Validation Dice Coefficient: {dice}')


In [ ]:
def visualize_predictions(model, x_val, y_val):
    predictions = model.predict(x_val)
    plt.figure(figsize=(20, 10))
    for i in range(5):
        plt.subplot(3, 5, i + 1)
        plt.imshow(x_val[i])
        plt.title('Input Image')
        plt.axis('off')

        plt.subplot(3, 5, i + 6)
        plt.imshow(y_val[i].squeeze(), cmap='gray')
        plt.title('True Mask')
        plt.axis('off')

        plt.subplot(3, 5, i + 11)
        plt.imshow(predictions[i].squeeze(), cmap='gray')
        plt.title('Predicted Mask')
        plt.axis('off')
    plt.show()

visualize_predictions(model, x_val, y_val)


In [ ]:
# Пример для обучения простой модели FCN
def build_simple_fcn_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(256, 256, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2,
